# 공공데이터포털에서 API를 통해 개인사업자재무정보를 가져옴

In [1]:
import pandas as pd
from pandas import json_normalize # json 형태의 데이터를 DataFrame으로 변환해주는 패키지
import requests # API 정보를 가져오기 위한 패키지
import json # json 형태의 데이터를 읽기 위한 패키지

In [2]:
# 공공데이터포털에 API 형태의 데이터를 요청(requests.get())
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBFinanceInfoService/getFnafInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=1&numOfRows=5000&resultType=json', verify = False)
# API로 받아온 데이터의 내용을 추출해 contents에 저장
contents = requestData.text

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# json 패키지의 loads()함수를 이용해 파이썬이 읽을 수 있는 자료형으로 만듦
data = json.loads(contents)

In [4]:
data

# 내용을 확인해보면 파이썬의 딕셔너리 형태와 유사
# 가져올 데이터의 형태를 확인

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'},
  'body': {'numOfRows': 5000,
   'pageNo': 1,
   'totalCount': 120972,
   'items': {'item': [{'basYm': '202208',
      'rprSexNm': '남성',
      'rprAggrNm': '50대',
      'estbYr': '2003',
      'bizAreaNm': '강원도 강릉시',
      'bizBzcCd': '42',
      'bizBzcCdNm': '전문직별 공사업',
      'empeCntNm': '1명 이상 5명 미만',
      'fnafBasYr': '2021',
      'cptlAmt': '181000000',
      'saleAmt': '1096000000',
      'bzopPftAmt': '99000000',
      'crtmNpfAmt': '86000000',
      'astTsumAmt': '841000000',
      'debtTsumAmt': '524000000'},
     {'basYm': '202208',
      'rprSexNm': '남성',
      'rprAggrNm': '50대',
      'estbYr': '1999',
      'bizAreaNm': '강원도 강릉시',
      'bizBzcCd': '47',
      'bizBzcCdNm': '소매업; 자동차 제외',
      'empeCntNm': '0명',
      'fnafBasYr': '2018',
      'cptlAmt': '23000000',
      'saleAmt': '1096000000',
      'bzopPftAmt': '74000000',
      'crtmNpfAmt': '75000000',
      'astTsumAmt': '77000000',

In [21]:
# 가져올 데이터의 형태를 확인하면,
# {'response' : {'header' : {}, 'body' : {}, 'items' : {'item' : {}}}} 의  4중 구조임을 알 수 있음
stat = data['response']['body']['items']['item']

In [22]:
# json_normalize()함수를 통해 DataFrame으로 변환
df = json_normalize(stat)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   basYm        5000 non-null   object
 1   rprSexNm     5000 non-null   object
 2   rprAggrNm    5000 non-null   object
 3   estbYr       5000 non-null   object
 4   bizAreaNm    5000 non-null   object
 5   bizBzcCd     5000 non-null   object
 6   bizBzcCdNm   5000 non-null   object
 7   empeCntNm    5000 non-null   object
 8   fnafBasYr    5000 non-null   object
 9   cptlAmt      5000 non-null   object
 10  saleAmt      5000 non-null   object
 11  bzopPftAmt   5000 non-null   object
 12  crtmNpfAmt   5000 non-null   object
 13  astTsumAmt   5000 non-null   object
 14  debtTsumAmt  5000 non-null   object
dtypes: object(15)
memory usage: 586.1+ KB


In [23]:
df.columns

Index(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd',
       'bizBzcCdNm', 'empeCntNm', 'fnafBasYr', 'cptlAmt', 'saleAmt',
       'bzopPftAmt', 'crtmNpfAmt', 'astTsumAmt', 'debtTsumAmt'],
      dtype='object')

In [24]:
# rename()함수를 통해 영어 약어로 되어 있던 column명들을 원하는 이름으로 바꿔줌
# inplace = True 옵션을 설정하면 원래 DataFrame 내의 값을 바꿈
# inplace = False(default)일 때에는 새로운 객체를 통해 저장해줘야 함(ex. new_df = df.rename())
df.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'fnafBasYr' : '재무기준년도', 'cptlAmt' : '자본금액', 'saleAmt' : '매출금액',
       'bzopPftAmt' : '영업이익', 'crtmNpfAmt' : '당기순이익', 'astTsumAmt' : '자산총합계금액', 'debtTsumAmt' : '부채총합계금액'}, inplace = True)

In [25]:
# column명들이 모두 바뀌었음을 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   기준년월     5000 non-null   object
 1   대표자성별명   5000 non-null   object
 2   대표자연령대명  5000 non-null   object
 3   설립년도     5000 non-null   object
 4   사업지역명    5000 non-null   object
 5   사업업종코드   5000 non-null   object
 6   사업업종명    5000 non-null   object
 7   종업원수구분명  5000 non-null   object
 8   재무기준년도   5000 non-null   object
 9   자본금액     5000 non-null   object
 10  매출금액     5000 non-null   object
 11  영업이익     5000 non-null   object
 12  당기순이익    5000 non-null   object
 13  자산총합계금액  5000 non-null   object
 14  부채총합계금액  5000 non-null   object
dtypes: object(15)
memory usage: 586.1+ KB


# 다른 페이지의 정보를 가져오기 위한 코드

In [26]:
# requests.get() 내의 주소 중 pageNo 값만 변경
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBFinanceInfoService/getFnafInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=10&numOfRows=5000&resultType=json', verify = False)

contents = requestData.text

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [27]:
data = json.loads(contents)
stat = data['response']['body']['items']['item']

In [28]:
df2 = json_normalize(stat)
df2.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'fnafBasYr' : '재무기준년도', 'cptlAmt' : '자본금액', 'saleAmt' : '매출금액',
       'bzopPftAmt' : '영업이익', 'crtmNpfAmt' : '당기순이익', 'astTsumAmt' : '자산총합계금액', 'debtTsumAmt' : '부채총합계금액'}, inplace = True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   기준년월     5000 non-null   object
 1   대표자성별명   5000 non-null   object
 2   대표자연령대명  5000 non-null   object
 3   설립년도     5000 non-null   object
 4   사업지역명    5000 non-null   object
 5   사업업종코드   5000 non-null   object
 6   사업업종명    5000 non-null   object
 7   종업원수구분명  5000 non-null   object
 8   재무기준년도   5000 non-null   object
 9   자본금액     5000 non-null   object
 10  매출금액     5000 non-null   object
 11  영업이익     5000 non-null   object
 12  당기순이익    5000 non-null   object
 13  자산총합계금액  5000 non-null   object
 14  부채총합계금액  5000 non-null   object
dtypes: object(15)
memory usage: 586.1+ KB


# 두 페이지에서 가져온 데이터를 합쳐 csv파일로 저장

In [29]:
# 두 개의 DataFrame의 column 형태가 같으므로 pd.concat()함수를 통해 바로 합침
df3 = pd.concat([df, df2])
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   기준년월     10000 non-null  object
 1   대표자성별명   10000 non-null  object
 2   대표자연령대명  10000 non-null  object
 3   설립년도     10000 non-null  object
 4   사업지역명    10000 non-null  object
 5   사업업종코드   10000 non-null  object
 6   사업업종명    10000 non-null  object
 7   종업원수구분명  10000 non-null  object
 8   재무기준년도   10000 non-null  object
 9   자본금액     10000 non-null  object
 10  매출금액     10000 non-null  object
 11  영업이익     10000 non-null  object
 12  당기순이익    10000 non-null  object
 13  자산총합계금액  10000 non-null  object
 14  부채총합계금액  10000 non-null  object
dtypes: object(15)
memory usage: 1.2+ MB


In [31]:
# to_csv 함수를 통해 csv파일로 저장
# sep = ',' -> csv(comma seperated values)이므로 구분자는 ','
# na_rep = 'NaN' -> DataFrame의 결측값을 어떻게 저장할지 지정해주는 옵션
# encoding = 'utf-8-sig' -> csv파일을 저장할 때 한글이 있다면 'utf-8-sig'로 지정해줘야 함
df3.to_csv('C:\\Users\\DMC CONET\\Documents\\python\\project1\\data\\개인사업자재무정보.csv', sep = ',', na_rep = 'NaN', encoding = 'utf-8-sig')